In [16]:
#!pip install yfinance
#!pip install chart_studio
#!pip install cufflinks

In [17]:
import yfinance as yf #Worked as of 10/03/20
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

#for å regne ut beta
import math
import numpy as np
import matplotlib.pyplot as plt
#import pandas_datareader as web
from scipy import stats
import seaborn as sns
import random
import configparser as cp
import scipy.optimize as sco #for optimering

#For plotting
import cufflinks as cf #Plotly wrapper
cf.set_config_file(offline=True) #Plotting modellen blir offline med denne metoden. 

#Yfinance 2
from yahoofinancials import YahooFinancials

In [190]:
#Edit yfinance file.
yf.__file__

'/Users/klaskan/opt/anaconda3/lib/python3.7/site-packages/yfinance/__init__.py'

## Finn aksjer 
Her vil jeg ta i bruk:
1. P/E Ratio (price-to-earning ratio).
2. Price/Earnings-to-Growth – PEG Ratio 
3. ROE Ratio (return on equity ratio).
4. P/B Ratio (price to book ratio)
5. Debt/ Equity Ratio.
6. Profit Margins


In [141]:
#Scrap data / akkumulere all finansiell data
s_og_p500 = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
soup = bs(s_og_p500,'lxml')
stock_ticker = []
company = []
industry = []
beta = [] 
long_info = []

pe_ratio = []
PEG_ratio = []
ROE_ratio = []
PB_ratio = []
DE_ratio = []
profit_margins = []



for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
  table_data = items.find_all(['th','td'])
  try:
    stock_ticker.append(table_data[0].a.text) #ticker
    company.append(table_data[1].a.text) #Navn bedrfit
    industry.append(table_data[4].text) #industri
    try:
      beta.append(yf.Ticker(table_data[0].a.text).info['beta']) #beta
    except:
      beta.append(0)  
    try:
      long_info.append(yf.Ticker(table_data[0].a.text).info['longBusinessSummary']) #Full info
    except:
      long_info.append('None') 
    try:
      PB_ratio.append(yf.Ticker(table_data[0].a.text).info['bookValue'])
    except:
      PB_ratio.append('None')
    try:
      pe_ratio.append(yf.Ticker(table_data[0].a.text).info['trailingPE'])
    except:
      pe_ratio.append('None')
    try:
      ROE_ratio.append(yf.Ticker(table_data[0].a.text).financials.iat[4, 0] / yf.Ticker(table_data[0].a.text).balance_sheet.iat[3,0] )
    except:
      ROE_ratio.append('None')
    try:
      PEG_ratio.append(yf.Ticker(table_data[0].a.text).info['pegRatio'])
    except:
      PEG_ratio.append('None')
    try:
      DE_ratio.append(yf.Ticker(table_data[0].a.text).balance_sheet.iat[0,0] / yf.Ticker(table_data[0].a.text).balance_sheet.iat[1,0])
    except:
      DE_ratio.append('None')    
    try:
      profit_margins.append(yf.Ticker(table_data[0].a.text).info['profitMargins'])
    except:
      profit_margins.append('None')
    
  except:
    pass

#for comp in company:
#  try:
#    sentiment.append(stock_sentiment(comp))# sentiment analyse
#  except:
#    sentiment.append(0)

data_to_df = {'Ticker': stock_ticker, 'Company Name': company, 'Industry' : industry, 'Beta' : beta, 'Business Summary' : long_info, 'P/E ratio': pe_ratio, 'PEG ratio': PEG_ratio, 'ROE ratio': ROE_ratio, 'P/B ratio': PB_ratio, 'D/E ratio' : DE_ratio, 'Profit Margin': profit_margins } #'Sentiment Analysis' : sentiment,
df = pd.DataFrame(data_to_df)

#pandas settings
pd.set_option('display.max_rows', None)
#pd.set_option('max_colwidth',  None)


sorted_df = df.sort_values(by='Industry', ascending=True)

#sorted_df.to_csv('data.csv') 
#files.download('data.csv')

sorted_df


,Ticker,Company Name,Industry,Beta,Business Summary,P/E ratio,PEG ratio,ROE ratio,P/B ratio,D/E ratio,Profit Margin
256,IPG,Interpublic Group,Advertising,1.180746,"The Interpublic Group of Companies, Inc. provi...",9.69048,-44.95,0.23632,7.173,1.03786,0.07606
360,OMC,Omnicom Group,Advertising,0.000000,None,None,None,None,None,None,None
52,ARNC,Arconic Inc.,Aerospace & Defense,0.000000,None,None,None,None,None,None,None
284,LHX,L3Harris Technologies,Aerospace & Defense,0.000000,None,None,None,None,None,None,None
397,RTX,Raytheon Co.,Aerospace & Defense,0.000000,None,None,None,None,None,None,None
449,TDG,TransDigm Group,Aerospace & Defense,1.529142,"TransDigm Group Incorporated designs, produces...",33.2331,2.07,-0.307357,-28.071,1.29711,0.17519
297,LMT,Lockheed Martin Corp.,Aerospace & Defense,0.959397,"Lockheed Martin Corporation, a security and ae...",16.902,1.75,141.591,11.168,0.0839552,0.10416
210,GD,General Dynamics,Aerospace & Defense,NaN,None,11.7705,1.7,0.25661,None,0.761764,None
350,NOC,Northrop Grumman,Aerospace & Defense,0.912546,Northrop Grumman Corporation operates as a sec...,16.8092,1.38,10.1861,52.541,0.0322281,0.10308
240,HWM,Howmet Aerospace,Aerospace & Defense,0.000000,None,None,None,None,None,None,None


In [212]:
def getDataframe(portfolio):
    stock_ticker = []
    company = []
    industry = []
    beta = [] 
    long_info = []

    pe_ratio = []
    PEG_ratio = []
    ROE_ratio = []
    PB_ratio = []
    DE_ratio = []
    profit_margins = []
    
    for item in portfolio:
        try:
            stock_ticker.append(item) #ticker
            company.append(yf.Ticker(item).info['shortName']) #Navn bedrfit
            industry.append(yf.Ticker(item).info['industry']) #industri
            try:
              beta.append(yf.Ticker(item).info['beta']) #beta
            except:
              beta.append(0)  
            try:
              long_info.append(yf.Ticker(item).info['longBusinessSummary']) #Full info
            except:
              long_info.append('None') 
            try:
              PB_ratio.append(yf.Ticker(item).info['bookValue'])
            except:
              PB_ratio.append('None')
            try:
              pe_ratio.append(yf.Ticker(item).info['trailingPE'])
            except:
              pe_ratio.append('None')
            try:
              ROE_ratio.append(yf.Ticker(item).financials.iat[4, 0] / yf.Ticker(item).balance_sheet.iat[3,0] )
            except:
              ROE_ratio.append('None')
            try:
              PEG_ratio.append(yf.Ticker(item).info['pegRatio'])
            except:
              PEG_ratio.append('None')
            try:
              DE_ratio.append(yf.Ticker(item).balance_sheet.iat[0,0] / yf.Ticker(item).balance_sheet.iat[1,0])
            except:
              DE_ratio.append('None')    
            try:
              profit_margins.append(yf.Ticker(item).info['profitMargins'])
            except:
              profit_margins.append('None')
    
        except:
            pass
    
    data_to_df = {'Ticker': stock_ticker, 'Company Name': company, 'Industry' : industry, 'Beta' : beta,  'P/E ratio': pe_ratio, 'PEG ratio': PEG_ratio, 'ROE ratio': ROE_ratio, 'P/B ratio': PB_ratio, 'D/E ratio' : DE_ratio, 'Profit Margin': profit_margins, 'Business Summary' : long_info} #'Sentiment Analysis' : sentiment,
    df = pd.DataFrame(data_to_df)
    sorted_df = df.sort_values(by='Industry', ascending=True)
    return sorted_df
    

test1234 = getDataframe(portfolio)
    

In [229]:
test1234

,Ticker,Company Name,Industry,Beta,P/E ratio,PEG ratio,ROE ratio,P/B ratio,D/E ratio,Profit Margin,Business Summary
1,NOC,Northrop Grumman Corporation,Aerospace & Defense,0.833660,25.631618,1.38,7.09148,52.541,0.032228,0.06643,None
11,FITB,Fifth Third Bancorp,Banks—Regional,1.668631,5.216217,0.87,0.129265,27.412,0.331759,0.31826,Fifth Third Bancorp operates as a diversified ...
13,PEP,"Pepsico, Inc.",Beverages—Non-Alcoholic,0.578912,25.171154,4.14,0.494657,10.630,4.128410,0.10890,"PepsiCo, Inc. operates as a food and beverage ..."
4,ALXN,"Alexion Pharmaceuticals, Inc.",Biotechnology,1.375900,9.022430,0.67,0.213302,50.934,0.379831,0.48172,"Alexion Pharmaceuticals, Inc. develops and com..."
7,APD,"Air Products and Chemicals, Inc",Chemicals,0.863785,25.407217,1.96,0.159224,52.366,0.391727,0.21097,"Air Products and Chemicals, Inc. provides atmo..."
2,PYPL,"PayPal Holdings, Inc.",Credit Services,0.938851,51.198070,1.69,0.145632,14.395,0.049910,0.13836,"PayPal Holdings, Inc. operates as a technology..."
10,NFLX,"Netflix, Inc.",Entertainment,1.031434,96.058105,1.43,0.349721,17.279,0.557081,0.09262,"Netflix, Inc. provides subscription streaming ..."
5,ICE,Intercontinental Exchange Inc.,Financial Data & Stock Exchanges,0.502292,24.906433,2.27,0.112025,31.125,0.899251,0.37159,"Intercontinental Exchange, Inc. operates regul..."
0,NKE,"Nike, Inc.",Footwear & Accessories,0.820563,31.177555,3.27,None,5.817,0.039509,0.10462,"NIKE, Inc., together with its subsidiaries, de..."
9,GOOGL,Alphabet Inc.,Internet Content & Information,1.045841,24.620346,5.31,1.54984,292.651,0.026576,0.21218,Alphabet Inc. provides online advertising serv...


In [150]:
sorted_df.to_csv('/Users/klaskan/Downloads/test123.csv', index=False) 


In [157]:
portfolio = ['NKE', #Nike
             'NOC', #Northrop Grumman
             'PYPL', #PayPal
             'ADBE', #Adobe Inc 
             'ALXN', #Alexion Pharmaceuticals
             'ICE', #Intercontinental Exchange 
             'ABMD', #ABIOMED Inc
             'APD', #Air Products & Chemicals Inc
             'T', #AT&T Inc.
             'GOOGL', #Alphabet Inc Class A
             'NFLX', #Netflix inc
             'FITB', #Fifth Third Bancorp
             'AVGO',#Broadcom inc.
             'PEP',#PepsiCo Inc
             'AWK' ]#American Water Works Company Inc

In [158]:
port_data = yf.Tickers(portfolio)

In [159]:
hist_close_data =port_data.history(start='2018-01-01', end='2020-01-01')['Close']

[*********************100%***********************]  15 of 15 completed


In [230]:

hist_close_data

,ABMD,ADBE,ALXN,APD,AVGO,AWK,FITB,GOOGL,ICE,NFLX,NKE,NOC,PEP,PYPL,T
Date,,,,,,,,,,,,,,,
2018-01-02,192.490005,177.699997,124.050003,156.573700,244.914810,85.360283,28.167418,1073.209961,67.840370,201.070007,61.950970,294.740417,110.229088,73.839996,33.197472
2018-01-03,195.820007,181.039993,124.949997,157.425095,247.593185,84.756577,28.250725,1091.520020,68.869881,205.050003,61.941216,296.101929,109.939651,76.750000,32.430851
2018-01-04,199.250000,183.220001,122.690002,158.030533,247.675720,83.951653,28.352545,1095.760010,70.171310,205.630005,61.902187,297.936584,110.481171,76.730003,32.732330
2018-01-05,202.320007,185.339996,122.800003,159.354950,249.143311,83.127563,28.426598,1110.290039,70.754051,209.990005,62.429100,297.617950,110.798622,78.699997,32.852921
2018-01-08,207.800003,185.039993,119.699997,159.525238,249.739563,83.846245,28.417343,1114.209961,71.016296,212.050003,62.985283,297.656555,110.163719,79.050003,32.947678
2018-01-09,209.759995,186.699997,121.800003,159.932007,246.281555,83.089226,28.926445,1112.790039,71.919540,209.309998,62.536427,299.713318,110.247765,79.190002,32.720753
2018-01-10,208.139999,187.110001,121.459999,159.260376,241.080704,82.121407,29.231909,1110.140015,72.317741,212.520004,62.663284,297.028931,109.687561,79.370003,31.961418
2018-01-11,210.149994,188.919998,122.480003,160.792892,241.695267,81.201492,29.491091,1112.050049,72.240044,217.240005,62.731586,298.902191,109.127365,79.750000,31.839231
2018-01-12,215.110001,195.050003,122.709999,160.963196,242.419891,77.454773,29.944653,1130.650024,72.628540,221.229996,63.102371,304.309540,109.594185,80.540001,32.205807


In [160]:
#Drop all None values
hist_close_data.dropna(inplace=True)

In [198]:
#Visualiser det normaliserte timeseries dataen. 6.12 .normalize()
hist_close_data.normalize().iplot(kind='lines')

Regner ut log return på en vektorisert måte for alle dagene tilgjengelig.

In [162]:
hist_close_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 503 entries, 2018-01-02 to 2019-12-31
Data columns (total 15 columns):
ABMD     503 non-null float64
ADBE     503 non-null float64
ALXN     503 non-null float64
APD      503 non-null float64
AVGO     503 non-null float64
AWK      503 non-null float64
FITB     503 non-null float64
GOOGL    503 non-null float64
ICE      503 non-null float64
NFLX     503 non-null float64
NKE      503 non-null float64
NOC      503 non-null float64
PEP      503 non-null float64
PYPL     503 non-null float64
T        503 non-null float64
dtypes: float64(15)
memory usage: 62.9 KB


## Returns

In [163]:
#Regner ut log avkasting
log_returns = np.log(hist_close_data / hist_close_data.shift(1))

In [164]:
log_returns

,ABMD,ADBE,ALXN,APD,AVGO,AWK,FITB,GOOGL,ICE,NFLX,NKE,NOC,PEP,PYPL,T
Date,,,,,,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.017152,0.018621,0.007229,0.005423,0.010877,-0.007098,0.002953,0.016917,0.015061,0.019601,-0.000157,0.004609,-0.002629,0.038653,-0.023364
2018-01-04,0.017364,0.011970,-0.018253,0.003839,0.000333,-0.009542,0.003598,0.003877,0.018721,0.002825,-0.000630,0.006177,0.004914,-0.000261,0.009253
2018-01-05,0.015290,0.011504,0.000896,0.008346,0.005908,-0.009865,0.002608,0.013173,0.008270,0.020981,0.008476,-0.001070,0.002869,0.025350,0.003677
2018-01-08,0.026725,-0.001620,-0.025568,0.001068,0.002390,0.008608,-0.000326,0.003524,0.003700,0.009762,0.008870,0.000130,-0.005747,0.004437,0.002880
2018-01-09,0.009388,0.008931,0.017392,0.002547,-0.013943,-0.009070,0.017757,-0.001275,0.012639,-0.013006,-0.007152,0.006886,0.000763,0.001769,-0.006911
2018-01-10,-0.007753,0.002194,-0.002795,-0.004208,-0.021344,-0.011716,0.010505,-0.002384,0.005521,0.015220,0.002026,-0.008997,-0.005094,0.002270,-0.023480
2018-01-11,0.009611,0.009627,0.008363,0.009577,0.002546,-0.011265,0.008827,0.001719,-0.001075,0.021967,0.001089,0.006287,-0.005120,0.004776,-0.003830
2018-01-12,0.023328,0.031932,0.001876,0.001059,0.002994,-0.047239,0.015263,0.016588,0.005363,0.018200,0.005893,0.017929,0.004269,0.009857,0.011448


In [165]:
log_returns.iplot(kind='histogram', subplots=True)
#Her kan du se log returns etter Ticker
#Slik ser altså da frekvens distribusjonen ut. 
#Som vi allerede vet fra empiriske bevis skal ikke disse være normaldistribuert.

In [166]:
"""
I mean-variance portfolio teori spiller den gjennomsnittelige avkastninge
en viktig rolle siden de brukes til å aproksimere den forventede
avkastningen. Så det vi har sett i tidligere tid vil vi anta er en god 
aproksimasjon på hva vi kan forvente å se i fremtiden. 
"""
#Dermed er daily mean returns gitt ved:
log_returns.mean()


ABMD    -0.000241
ADBE     0.001232
ALXN    -0.000273
APD      0.000796
AVGO     0.000474
AWK      0.000718
FITB     0.000142
GOOGL    0.000441
ICE      0.000612
NFLX     0.000948
NKE      0.000974
NOC      0.000300
PEP      0.000415
PYPL     0.000761
T        0.000264
dtype: float64

In [167]:
#Denne daglige log returns kan vi skalere opp et "tading year"
log_returns.mean() * 252

ABMD    -0.060631
ADBE     0.310442
ALXN    -0.068856
APD      0.200549
AVGO     0.119494
AWK      0.180905
FITB     0.035908
GOOGL    0.111221
ICE      0.154099
NFLX     0.238829
NKE      0.245510
NOC      0.075710
PEP      0.104554
PYPL     0.191662
T        0.066465
dtype: float64

In [168]:
#Her kan vi enklere ser hvem som har gjort det best.
#Ikke alle har en positiv akastning engang.
(log_returns.mean() * 252).iplot(kind = 'bar')

## Volatilities
Analyserer volatiliteten.
Den historiske volatiliteten også kjent som standardavviket av avkastningen spiller en vikrig rolle, men i porfoliostatistics kalkuleringene brukes ikke disse direkte. Vi kommer heller til å bruke covariance matrisen i det tilfellet. 

In [169]:
#Historisk standardavvik
log_returns.std()

ABMD     0.032113
ADBE     0.019010
ALXN     0.023861
APD      0.012309
AVGO     0.020646
AWK      0.010625
FITB     0.015954
GOOGL    0.016413
ICE      0.011563
NFLX     0.025716
NKE      0.015921
NOC      0.015607
PEP      0.010280
PYPL     0.018872
T        0.013321
dtype: float64

In [170]:
(log_returns.std() * math.sqrt(252)) #Trading year volatilitet.

ABMD     0.509779
ADBE     0.301771
ALXN     0.378779
APD      0.195404
AVGO     0.327751
AWK      0.168666
FITB     0.253258
GOOGL    0.260554
ICE      0.183550
NFLX     0.408230
NKE      0.252742
NOC      0.247751
PEP      0.163189
PYPL     0.299586
T        0.211465
dtype: float64

In [171]:
#Plotter denne volatiliteten.
(log_returns.std() * math.sqrt(252)).iplot(kind='bar', colors='green')
"""
Figuren ser ut til å stemme med prinsippet om at høyere forventet 
avkastning ser ut til å ha den høyere volatilitet kan vi forvente å se.
"""

'\nFiguren ser ut til å stemme med prinsippet om at høyere forventet \navkastning ser ut til å ha den høyere volatilitet kan vi forvente å se.\n'

## Portefølje statistics 
Her vil jeg bare anta at vi har et portegølje med aksjer som alle har lik vektlegging. Hver aksje tar 6.66% av porteføljen.

In [33]:
100/15

6.666666666666667

In [172]:
weights = len(log_returns.columns) * [1/len(log_returns.columns)]
weights

[0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667]

In [173]:
"""
Den forventede poreføljeavkastingen ifølge mean-variance portfolio teori
er dot produktet av den forventede avkastningen og vektleggingen. 
"""
np.dot(log_returns.mean() * 252, weights)

0.12705747187573846

In [174]:
def portfolio_return(symbols, weights):
    return np.dot(log_returns[symbols].mean() * 252, weights)

In [175]:
portfolio_return(log_returns.columns, weights)

0.12705747187573846

In [176]:
#Kalkuler covaraiance matrisen 

hist_close_data.cov() * 252 #Covarinase årlig basis. 


,ABMD,ADBE,ALXN,APD,AVGO,AWK,FITB,GOOGL,ICE,NFLX,NKE,NOC,PEP,PYPL,T
ABMD,1.517011e+06,-182270.379364,100554.032411,-430462.785142,-420563.659432,-224911.524724,-4014.424581,-5.094233e+05,-114617.337437,481607.044267,-71918.988840,-367543.781427,-192955.263721,-128150.053477,-46904.764139
ADBE,-1.822704e+05,250697.730799,-1038.225507,196734.073199,178356.954496,106785.069904,-2846.470277,4.279059e+05,51836.121978,132085.331514,66493.791749,86560.210384,76896.127735,88669.538673,10402.471050
ALXN,1.005540e+05,-1038.225507,26461.854705,-18622.870845,-6491.428905,-11792.608895,-16.849543,1.941053e+03,-8678.595461,68552.430876,-2316.703327,-38155.562340,-8443.587954,1412.650892,-3859.047578
APD,-4.304628e+05,196734.073199,-18622.870845,226484.128234,209373.717710,116534.535936,-193.197044,3.650971e+05,57451.279940,-7054.515486,56002.479452,144925.709407,90038.247450,86933.107910,16733.687969
AVGO,-4.205637e+05,178356.954496,-6491.428905,209373.717710,271250.665066,107808.025233,-481.572366,3.819366e+05,49767.057226,7675.202113,57347.099802,92368.450632,83687.235320,84699.462576,15195.913281
AWK,-2.249115e+05,106785.069904,-11792.608895,116534.535936,107808.025233,67718.933617,-2405.166462,1.842653e+05,31785.349601,-8522.891857,32634.758538,59020.192893,50713.001308,47534.515791,7865.581716
FITB,-4.014425e+03,-2846.470277,-16.849543,-193.197044,-481.572366,-2405.166462,1193.786201,8.596579e+03,-435.593645,-2015.343154,-1070.555588,9478.248417,-1220.499149,-1839.991593,676.449522
GOOGL,-5.094233e+05,427905.865739,1941.053063,365097.129361,381936.561752,184265.259619,8596.578772,1.712016e+06,99609.326585,136088.520279,144541.848046,268563.353240,160747.828241,139247.924199,39464.613018
ICE,-1.146173e+05,51836.121978,-8678.595461,57451.279940,49767.057226,31785.349601,-435.593645,9.960933e+04,17163.922536,-13638.050204,15383.748207,40709.749777,24989.388650,21270.952875,4773.043403
NFLX,4.816070e+05,132085.331514,68552.430876,-7054.515486,7675.202113,-8522.891857,-2015.343154,1.360885e+05,-13638.050204,456004.721841,23573.425149,-78549.427815,-18885.398321,35664.753245,-12730.085317


In [177]:
#Den forventede portfoliog variansen.
#For å finne portefølje variansen må vi ta i bruk et dobblet 
#dot produkt/nøstet dot produkt. 
#Dette er den forventede portefølje variansen:
np.dot(weights, np.dot(log_returns.cov() * 252, weights))

0.028197634490246933

In [178]:
#Dermed er den forventede portefølje volatiliteten 
math.sqrt(np.dot(weights, np.dot(log_returns.cov() * 252, weights)))

0.16792151288696436

In [179]:
def portfolio_volatility(symbols, weights):
    return math.sqrt(np.dot(weights, np.dot(log_returns[symbols].cov() * 252, weights)))

In [180]:
portfolio_volatility(log_returns.columns, weights)

0.16792151288696436

## Simulasjon av portefølje komposisjonen

In [181]:
w = np.random.random((2500, len(portfolio)))

In [182]:
w[:5].round(2)

array([[0.49, 0.56, 0.06, 0.75, 0.24, 0.29, 0.33, 0.8 , 0.87, 0.25, 0.63,
        0.66, 0.62, 0.52, 0.12],
       [0.41, 0.7 , 0.89, 0.67, 0.26, 0.31, 0.21, 0.34, 0.22, 0.9 , 0.61,
        0.52, 0.54, 0.81, 0.19],
       [0.27, 0.46, 0.97, 0.54, 0.26, 0.32, 0.53, 0.69, 0.71, 0.92, 0.7 ,
        0.13, 0.01, 0.84, 0.95],
       [0.87, 0.96, 0.51, 0.98, 0.24, 0.22, 0.88, 0.57, 0.99, 0.67, 0.85,
        0.94, 0.96, 0.66, 0.38],
       [0.07, 0.46, 0.58, 0.11, 0.48, 0.16, 0.11, 0.94, 0.88, 0.97, 0.25,
        0.03, 0.27, 0.2 , 0.75]])

In [183]:
w = (w.T / w.sum(axis=1)).T  # normalization ...

In [184]:
w[:5].round(2)  # ... let's the random numbers add up to 100%

array([[0.07, 0.08, 0.01, 0.1 , 0.03, 0.04, 0.05, 0.11, 0.12, 0.04, 0.09,
        0.09, 0.09, 0.07, 0.02],
       [0.05, 0.09, 0.12, 0.09, 0.03, 0.04, 0.03, 0.04, 0.03, 0.12, 0.08,
        0.07, 0.07, 0.11, 0.02],
       [0.03, 0.06, 0.12, 0.07, 0.03, 0.04, 0.06, 0.08, 0.09, 0.11, 0.08,
        0.02, 0.  , 0.1 , 0.11],
       [0.08, 0.09, 0.05, 0.09, 0.02, 0.02, 0.08, 0.05, 0.09, 0.06, 0.08,
        0.09, 0.09, 0.06, 0.04],
       [0.01, 0.07, 0.09, 0.02, 0.08, 0.03, 0.02, 0.15, 0.14, 0.15, 0.04,
        0.01, 0.04, 0.03, 0.12]])

In [185]:
w.sum(axis=1)[:5] # ... let's the random numbers add up to 100%

array([1., 1., 1., 1., 1.])

In [186]:
mvp = [(portfolio_volatility(portfolio, weights),
       portfolio_return(portfolio, weights))
         for weights in w]

In [187]:
mvp = pd.DataFrame(np.array(mvp), columns=['volatility', 'return'])
mvp.iloc[:5]

,volatility,return
0,0.166974,0.136886
1,0.176320,0.142579
2,0.165037,0.141527
3,0.174961,0.130802
4,0.156739,0.125825


In [188]:
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

In [121]:
#test
port_data2 = yf.Ticker('AMZN')

In [122]:
port_data2.info

{'zip': '98109-5210',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 798000,
 'longBusinessSummary': 'Amazon.com, Inc. engages in the retail sale of consumer products and subscriptions in North America and internationally. The company operates through three segments: North America, International, and Amazon Web Services (AWS). It sells merchandise and content purchased for resale from third-party sellers through physical and online stores. The company also manufactures and sells electronic devices, including Kindle, Fire tablets, Fire TVs, Rings, and Echo and other devices; provides Kindle Direct Publishing, an online service that allows independent authors and publishers to make their books available in the Kindle Store; and develops and produces media content. In addition, it offers programs that enable sellers to sell their products on its Websites, as well as its stores; and programs that allow authors, musicians, filmmakers, skill and app developers, and others to publish a